# -Day 1- 搭建开发环境

 转链接https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432170937506ecfb2f6adf8e4757939732f3e32b781c000

# -Day 2- 编写Web App骨架

用aiohttp写一个基本的app.py

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import asyncio, os, json, time
from datetime import datetime

from aiohttp import web

In [3]:
def index(request):
    return web.Response(body=b'<h1>Awesome</h1>')

@asyncio.coroutine
def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('Get', '/', index)
    srv = yield from loop.create_server(app.make_handler(), '127.0.0.1', 9000)
    logging.info('server started at http://127.0.0.1:9000...')
    return srv

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

/home/dell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: loop argument is deprecated
  
/home/dell/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Application.make_handler(...) is deprecated, use AppRunner API instead
  
INFO:root:server started at http://127.0.0.1:9000...
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:02:08 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:02:26 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
INFO:aiohttp.access:127.0.0.1 [18/Dec/2018:12:04:02 +0000] "GET / HTTP/1.1" 200 166 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0"
